<a href="https://colab.research.google.com/github/psygorozco/ML/blob/main/LLM/RAG_HAIKU_YOUTUBE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qU \
    langchain==0.1.11 \
    langchain-core==0.1.30 \
    langchain-community==0.0.27 \
    langchain-anthropic==0.1.4 \
    langchainhub==0.1.15 \
    anthropic==0.19.1 \
    voyageai==0.2.1 \
    chromadb \
    youtube-transcript-api

In [5]:
import os
from langchain.document_loaders import YoutubeLoader

def download_youtube_transcripts(youtube_urls, output_dir=".", output_filenames=None):
    """
    Descarga las transcripciones de múltiples videos de YouTube y las guarda en formato de texto (.txt) en un directorio local.

    Args:
        youtube_urls (list): Lista de URLs de los videos de YouTube.
        output_dir (str, optional): Directorio donde se guardarán los archivos de texto. Por defecto, se usa el directorio actual.
        output_filenames (list, optional): Lista de nombres de los archivos de texto. Si no se proporciona, se usarán los IDs de los videos de YouTube.
    """
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Procesar cada URL de YouTube
    for i, youtube_url in enumerate(youtube_urls):
        # Cargar la transcripción del video de YouTube
        loader = YoutubeLoader.from_youtube_url(youtube_url, add_video_info=False, language=["es"])
        docs = loader.load()

        # Obtener el ID del video de YouTube
        video_id = loader.video_id

        # Establecer el nombre del archivo de salida
        if output_filenames is None:
            output_filename = f"{video_id}.txt"
        else:
            output_filename = output_filenames[i]
        output_path = os.path.join(output_dir, output_filename)

        # Guardar la transcripción en formato de texto
        with open(output_path, "w", encoding="utf-8") as f:
            for doc in docs:
                f.write(doc.page_content)

        print(f"Transcripción guardada en: {output_path}")

In [10]:
youtube_urls = [
    "https://youtu.be/3Smhpbid2YY?si=aDXDPr2ri5mWlzI3",
    "https://youtu.be/j8vYTDJGneU?si=278eEn6SAvFxze5S",
    "https://youtu.be/Z6gk59AeYkI?si=w_WTAX9Tkn132Rs4"
]
download_youtube_transcripts(youtube_urls, output_dir="transcripciones", output_filenames=["video1.txt", "video2.txt", "video3.txt"])

Transcripción guardada en: transcripciones/video1.txt
Transcripción guardada en: transcripciones/video2.txt
Transcripción guardada en: transcripciones/video3.txt


In [11]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Carga los textos
def load_all_documents(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            loader = TextLoader(file_path)
            documents.extend(loader.load())
    return documents

# Ejemplo de uso
directory_path = 'transcripciones'
all_documents = load_all_documents(directory_path)

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=20)
fragmentos = text_splitter.split_documents(all_documents)

In [ ]:
fragmentos

In [17]:
import os
from getpass import getpass

voyage_key = os.getenv("VOYAGE_API_KEY") or getpass("Voyage API key: ")

Voyage API key: ··········


In [18]:
from langchain_community.embeddings import VoyageEmbeddings

embed = VoyageEmbeddings(
    voyage_api_key=voyage_key, model="voyage-2"
)

In [20]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(fragmentos, embed, persist_directory="./chroma_db")

In [21]:
db

In [31]:
from langchain_anthropic import ChatAnthropic

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY") or getpass("Anthropic API key: ")

# chat completion llm


Anthropic API key: ··········


In [23]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever(search_type='similarity', search_kwargs={'k': 3})

HASTA AQUÍ


In [24]:
docs = retriever.get_relevant_documents("¿Qué es la depresión?")


In [28]:
print(docs[1].page_content)

en el vídeo de hoy veremos los principales síntomas y características de la depresión quédate conmigo hasta el final y descubrirás algunas claves


In [34]:
def consultas(vectores, pregunta):
    from langchain.chains import RetrievalQA

    llm = ChatAnthropic(
    anthropic_api_key=anthropic_api_key,
    model_name="claude-3-haiku-20240307",  # change "opus" -> "sonnet" for speed
    temperature=0.0
    )

    retriever = vectores.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.run(pregunta)
    return answer

In [36]:
consultas(db, "¿Cómo ayudar a alguién con depresión?")

'Según el contexto proporcionado, aquí hay 7 claves útiles para ayudar a alguien que sufre de depresión:\n\n1. Escuchar con atención y sin juzgar. Brindar un espacio seguro para que la persona pueda expresar sus sentimientos.\n\n2. Mostrar empatía y validar sus emociones. Evitar minimizar o desestimar lo que está sintiendo.\n\n3. Animar a la persona a buscar ayuda profesional, como un terapeuta o médico. Ofrecerse a acompañarla a las citas.\n\n4. Ayudar con tareas cotidianas que puedan resultar abrumadoras, como cocinar, limpiar o hacer las compras.\n\n5. Mantener contacto regular y hacer planes para salir juntos, aunque sea algo sencillo como dar un paseo.\n\n6. Ser paciente y no presionar a la persona a "mejorar" más rápido de lo que puede.\n\n7. Cuidar también de tu propio bienestar emocional para poder brindar un apoyo sostenido.\n\nLa clave es estar presente, ser comprensivo y alentar a la persona a buscar la ayuda que necesita. Si tienes más preguntas, no dudes en consultarme.'